# Supercharging Your Web Development with FastAPI

Welcome to the exciting world of FastAPI, where web development meets speed and simplicity! In this live Jupyter demonstration, we'll delve into the ins and outs of FastAPI, a modern Python web framework that's been gaining traction for its lightning-fast performance and intuitive API design.

Whether you're a seasoned developer looking to streamline your workflow or a newcomer eager to explore the latest tools in Python web development, this session is tailor-made for you. Join us as we explore FastAPI's key features, walk through code examples, and demonstrate how it can revolutionize the way you build web applications - all within the interactive environment of Jupyter.

Get ready to supercharge your development process with FastAPI - because in the fast-paced world of web development, every millisecond counts!

May 16, 2024

By baronpythonist (Byron): Intro generated from OpenAI's ChatGPT.

## Imports and app instantiation

In [40]:
from fastapi import FastAPI, status, HTTPException, Body
import nest_asyncio
from pydantic import BaseModel, Field, BeforeValidator, ConfigDict
from typing import Optional, Annotated
from enum import Enum
import uvicorn
from uuid import uuid1
from motor import motor_asyncio
import os

app = FastAPI()

try:
    mongo_url = os.environ["MONGODB_URL"]
except KeyError:
    mongo_url = None
if mongo_url is None:
    schedule_collection = None
else:
    client = motor_asyncio.AsyncIOMotorClient(os.environ["MONGODB_URL"])
    db = client.get_database("pycon")
    schedule_collection = db.get_collection("schedules")

## Example Pydantic Models

In [41]:
# Represents an ObjectId field in the database.
# It will be represented as a `str` on the model so that it can be serialized to JSON.
PyObjectId = Annotated[str, BeforeValidator(str)]

class PyTalk(BaseModel):
    id: Optional[PyObjectId] = Field(alias="_id", default=None)
    datetime: str
    topic: str
    speaker: str
    description: str | None = None
    model_config = ConfigDict(
        populate_by_name=True,
        arbitrary_types_allowed=True
    )

class PyTalkCollection(BaseModel):
    talks: list[PyTalk]

class EventType(str, Enum):
    meal = "meal"
    main = "main"
    keynote = "keynote"
    lightning = "lightning"

class PyEvent(BaseModel):
    id: Optional[PyObjectId] = Field(alias="_id", default=None)
    datetime: str
    title: str
    type: EventType
    model_config = ConfigDict(
        populate_by_name=True,
        arbitrary_types_allowed=True
    )

class PyEventCollection(BaseModel):
    events: list[PyEvent]

class PySchedule(BaseModel):
    id: Optional[PyObjectId] = Field(alias="_id", default=None)
    name: str
    talks: PyTalkCollection | None = None
    events: PyEventCollection | None = None
    model_config = ConfigDict(
        populate_by_name=True,
        arbitrary_types_allowed=True
    )

class PyScheduleCollection(BaseModel):
    schedules: list[PySchedule]

## Example Endpoints

In [42]:
class DataBase:
    all_schedules: PyScheduleCollection = PyScheduleCollection(schedules=[])

@app.get('/')
async def info_dump():
    """info_dump(): Simple hello world endpoint for PyCon.
    
    params: None
    output:
        JSON object:
            message: `str` = hello-world style message
    """
    return {'message': 'Hello PyCon US 2024!'}

@app.post('/schedule',
    response_description="Add new schedule",
    response_model=PySchedule,
    status_code=status.HTTP_201_CREATED,
    response_model_by_alias=False,
)
async def create_schedule(schedule: PySchedule = Body(...)):
    """create_schedule(schedule): Creates a schedule from a validated JSON object
    
    params:
        schedule: `PySchedule` = conference schedule defined by Pydantic model
    output:
        schedule: `PySchedule` = schedule + id
    """
    if schedule.id is None:
        schedule.id = PyObjectId(uuid1())
        if schedule_collection is not None:
            new_schedule = await schedule_collection.insert_one(
                schedule.model_dump(by_alias=True)
            )
            new_schedule.id = str(new_schedule.id)
            return new_schedule
        else:
            schedule.id = str(schedule.id)
            DataBase.all_schedules.schedules.append(schedule)
            return schedule
    else:
        raise HTTPException(status_code=status.HTTP_409_CONFLICT, detail="Schedule already exists!")

@app.get('/schedule/talks',
    response_description="Get all talks",
    response_model=PyTalkCollection,
    response_model_by_alias=False,
)
async def get_all_talks():
    """get_all_talks():
    
    params: None
    output:
        JSON object:
            all_talks: `list[PyTalk]` = list of talks with non-empty ids
    """
    all_talks = []
    if schedule_collection is not None:
        schedule_sets = await schedule_collection.find().to_list(100)
        all_schedules = schedule_sets[0].schedules if schedule_sets else []
    else:
        all_schedules = DataBase.all_schedules.schedules if DataBase.all_schedules else []
    for schedule in all_schedules:
        all_talks.extend(schedule.talks.talks if schedule.talks is not None else [])
    return PyTalkCollection(talks=all_talks)

@app.get('/schedule/{sch_id}/talks/{talk_id}',
    response_description="Get talk by Id",
    response_model=PyTalk,
    response_model_by_alias=False,
)
async def get_talk(sch_id: str, talk_id: str):
    """get_talk(sch_id, talk_id): Get specific talk by Id.
    
    params:
        sch_id: `str` = uuid of `PySchedule` as `str`
        talk_id: `str` = uuid of `PyTalk` as `str`
    output:

    """
    if schedule_collection is not None:
        schedule_sets = await schedule_collection.find().to_list(100)
        all_schedules = schedule_sets[0].schedules if schedule_sets else []
    else:
        all_schedules = DataBase.all_schedules.schedules if DataBase.all_schedules else []
    for schedule in all_schedules:
        if schedule.id == sch_id:
            all_talks = schedule.talks.talks if schedule.talks is not None else []
            for talk in all_talks:
                if talk.id == talk_id:
                    return talk
    raise HTTPException(status_code=status.HTTP_404_NOT_FOUND, detail="talk not found")

## Example API launcher

In [43]:
def launcher(app):
    def launch_app():
        nest_asyncio.apply()
        uvicorn.run(app)
    return launch_app

## Example Shedule Body (Copy into Swagger)

```json
{
  "name": "string",
  "talks": {
    "talks": [
      {
        "_id": "01",
        "datetime": "Friday @ 11:00 AM",
        "topic": "FastAPI Discussion and Demo",
        "speaker": "Byron Burks",
        "description": null
      }
    ]
  },
  "events": {
    "events": [
      {
        "_id": "11",
        "datetime": "Saturday @ 7:30 AM",
        "title": "Breakfast",
        "type": "meal"
      }
    ]
  }
}
```

Spin-up API; interupt this cell to take down API and continue (ignore `KeyboardInterruptError` message).

In [ ]:
runf = launcher(app)
runf()

## Playground area: write new API endpoints and try them out!